In [5]:
%matplotlib inline
import matplotlib.pyplot as plt
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
import numpy as np
import skimage.io as io
import pylab
pylab.rcParams['figure.figsize'] = (10.0, 8.0)
import mmcv

/data/anaconda3/envs/map/lib/python3.7/site-packages/skimage/io/manage_plugins.py:23: UserWarning: Your installed pillow version is < 7.1.0. Several security issues (CVE-2020-11538, CVE-2020-10379, CVE-2020-10994, CVE-2020-10177) have been fixed in pillow 7.1.0 or higher. We recommend to upgrade this library.
  from .collection import imread_collection_wrapper


# convert flag annotation to coco

In [16]:
import os.path as osp
import pdb
import json
from pathlib import Path
import os
def convert_flag_to_coco(ann_file, out_file=None, image_prefix=None):
    if isinstance(ann_file,list):
        data_infos = ann_file
    else:
        with open(ann_file) as f:
            data_infos = json.load(f)

    #data_infos = mmcv.load(ann_file)
    #pdb.set_trace()
    annotations = []
    images = []
    obj_count = 0
    for idx, v in enumerate(data_infos):
        filename = v['image_path']
        img_id = filename
        #print(img_id)
        if os.path.exists(filename):
            img_path = filename
        else:
            img_path = osp.join(image_prefix, filename)
        height, width = mmcv.imread(img_path).shape[:2]

        images.append(dict(
            id=img_path,
            file_name=filename,
            height=height,
            width=width))

        bboxes = []
        labels = []
        masks = []
        for _, obj in enumerate(v['annotation']):
            #assert not obj['bbox']
            #obj = obj['shape_attributes']
            #px = obj['all_points_x']
            #py = obj['all_points_y']
            #poly = [(x + 0.5, y + 0.5) for x, y in zip(px, py)]
            #poly = [p for x in poly for p in x]

            #x_min, y_min, x_max, y_max = (
            #    min(px), min(py), max(px), max(py))
            x_min,y_min,x_max,y_max = obj['bbox'][0][0],obj['bbox'][0][1],obj['bbox'][1][0],obj['bbox'][1][1],
            
            if x_max-x_min < 0 or y_max-y_min < 0:
                print(v)
                
            
            category = int(obj['class_id'])

            data_anno = dict(
                image_id=img_path,
                id=obj_count,
                category_id=category,
                bbox=[x_min, y_min, x_max - x_min, y_max - y_min],
                area=(x_max - x_min) * (y_max - y_min),
                segmentation=None,
                iscrowd=0)
            annotations.append(data_anno)
            obj_count += 1

    coco_format_json = dict(
        images=images,
        annotations=annotations,
        categories=[{'id':0, 'name': 'china'},
                    {'id':1, 'name': 'us'},
                   {'id':2, 'name': 'uk'},
                   {'id':3, 'name': 'russia'},
                   {'id':4, 'name': 'japan'},
                   {'id':5, 'name': 'france'},
                   {'id':6, 'name': 'german'},
                   {'id':7, 'name': 'italy'},
                   {'id':8, 'name': 'australia'},
                   {'id':9, 'name': 'korea'},
                   {'id':10, 'name': 'other'}])
    #mmcv.dump(coco_format_json, out_file)
    return coco_format_json

In [30]:
train_val = []
#for js in ['/data/flag/flag_ann_finaltest_update.json']:  #['/data/flag/flag_ann_train_all.json','/data/flag/flag_ann_val_all.json']:   #,'/data/flag/flag_ann_test_all.json']
for js in ['/data/flag/flag_ann_val_all.json']: #,'/data/flag/flag_ann_val_all.json','/data/flag/flag_ann_test_all.json']:
    with open(js) as f:
        js_file = json.load(f)
    train_val.extend(js_file)
len(train_val)

726

In [31]:
trainval_coco = convert_flag_to_coco(train_val,image_prefix='/data/flag/Images/images')
trainval_coco

{'images': [{'id': '/data/flag/Images/images/ffout-18/03256.jpg',
   'file_name': 'ffout-18/03256.jpg',
   'height': 360,
   'width': 640},
  {'id': '/data/flag/Images/images/ffout-17/01964.jpg',
   'file_name': 'ffout-17/01964.jpg',
   'height': 480,
   'width': 852},
  {'id': '/data/flag/Images/images/ffout-22/01705.jpg',
   'file_name': 'ffout-22/01705.jpg',
   'height': 480,
   'width': 854},
  {'id': '/data/flag/Images/images/ffout-10/00677.jpg',
   'file_name': 'ffout-10/00677.jpg',
   'height': 480,
   'width': 852},
  {'id': '/data/flag/Images/images/ffout-18/00408.jpg',
   'file_name': 'ffout-18/00408.jpg',
   'height': 360,
   'width': 640},
  {'id': '/data/flag/Images/images/ffout-14/00036.jpg',
   'file_name': 'ffout-14/00036.jpg',
   'height': 480,
   'width': 852},
  {'id': '/data/flag/Images/images/ffout-22/03495.jpg',
   'file_name': 'ffout-22/03495.jpg',
   'height': 480,
   'width': 854},
  {'id': '/data/flag/Images/images/ffout-17/07849.jpg',
   'file_name': 'ffout-1

In [32]:
with open('/data/flag/flag_coco_val_all.json','w') as f:
    json.dump(trainval_coco,f)

In [8]:
test_coco = convert_flag_to_coco('/data/flag/flag_ann_test_all.json',image_prefix='/data/flag/Images/images')
test_coco

ffout-17/03286.jpg
ffout-10/04458.jpg
ffout-17/02513.jpg
ffout-18/01808.jpg
ffout-17/01613.jpg
ffout-17/03065.jpg
ffout-17/05970.jpg
ffout-10/01944.jpg
ffout-22/01174.jpg
ffout-6/00852.jpg
ffout-17/05948.jpg
ffout-6/01611.jpg
ffout-6/01394.jpg
ffout-10/04133.jpg
ffout-17/00116.jpg
ffout-17/03781.jpg
ffout-6/00720.jpg
ffout-17/00627.jpg
ffout-17/04220.jpg
ffout-13/00547.jpg
ffout-18/00087.jpg
ffout-18/01829.jpg
ffout-22/01599.jpg
ffout-17/06522.jpg
ffout-18/01177.jpg
ffout-17/05602.jpg
ffout-2/01117.jpg
ffout-6/02908.jpg
ffout-17/02728.jpg
ffout-6/00932.jpg
ffout-7/00053.jpg
ffout-18/00600.jpg
ffout-2/01382.jpg
ffout-10/00130.jpg
ffout-22/03766.jpg
ffout-12/00094.jpg
ffout-18/01457.jpg
ffout-22/01791.jpg
ffout-4/00012.jpg
ffout-22/03565.jpg
ffout-18/01754.jpg
ffout-18/02792.jpg
ffout-6/02020.jpg
ffout-2/01345.jpg
ffout-2/01872.jpg
ffout-17/08270.jpg
ffout-5/00054.jpg
ffout-18/03042.jpg
ffout-6/01050.jpg
ffout-18/00989.jpg
ffout-8/00063.jpg
ffout-5/00050.jpg
ffout-2/02645.jpg
ffout-17/05

ffout-17/06864.jpg
ffout-17/07439.jpg
ffout-17/06339.jpg
ffout-17/06895.jpg
ffout-11/00015.jpg
ffout-17/04592.jpg
ffout-13/00354.jpg
ffout-17/00850.jpg
ffout-20/00107.jpg
ffout-18/02713.jpg
ffout-10/03260.jpg
ffout-17/00529.jpg
ffout-2/01317.jpg
ffout-22/03479.jpg
ffout-17/02307.jpg
ffout-2/01283.jpg
ffout-18/01579.jpg
ffout-2/00522.jpg
ffout-17/02864.jpg
ffout-22/03606.jpg
ffout-2/00371.jpg
ffout-17/07178.jpg
ffout-17/07027.jpg
ffout-2/01279.jpg
ffout-17/02692.jpg
ffout-10/02917.jpg
ffout-10/01829.jpg
ffout-2/00616.jpg
ffout-6/01516.jpg
ffout-17/02058.jpg
ffout-6/01969.jpg
ffout-17/03577.jpg
ffout-22/01284.jpg
ffout-2/01960.jpg
ffout-10/00309.jpg
ffout-17/02012.jpg
ffout-10/01875.jpg
ffout-14/00355.jpg
ffout-22/03830.jpg
ffout-18/01874.jpg
ffout-17/00587.jpg
ffout-17/05550.jpg
ffout-18/01705.jpg
ffout-14/00066.jpg
ffout-17/06184.jpg
ffout-14/00439.jpg
ffout-17/02772.jpg
ffout-17/03534.jpg
ffout-22/03508.jpg
ffout-17/06451.jpg
ffout-17/06206.jpg
ffout-17/07138.jpg
ffout-6/00923.jpg
ffo

{'images': [{'id': 'ffout-17/03286.jpg',
   'file_name': 'ffout-17/03286.jpg',
   'height': 480,
   'width': 852},
  {'id': 'ffout-10/04458.jpg',
   'file_name': 'ffout-10/04458.jpg',
   'height': 480,
   'width': 852},
  {'id': 'ffout-17/02513.jpg',
   'file_name': 'ffout-17/02513.jpg',
   'height': 480,
   'width': 852},
  {'id': 'ffout-18/01808.jpg',
   'file_name': 'ffout-18/01808.jpg',
   'height': 360,
   'width': 640},
  {'id': 'ffout-17/01613.jpg',
   'file_name': 'ffout-17/01613.jpg',
   'height': 480,
   'width': 852},
  {'id': 'ffout-17/03065.jpg',
   'file_name': 'ffout-17/03065.jpg',
   'height': 480,
   'width': 852},
  {'id': 'ffout-17/05970.jpg',
   'file_name': 'ffout-17/05970.jpg',
   'height': 480,
   'width': 852},
  {'id': 'ffout-10/01944.jpg',
   'file_name': 'ffout-10/01944.jpg',
   'height': 480,
   'width': 852},
  {'id': 'ffout-22/01174.jpg',
   'file_name': 'ffout-22/01174.jpg',
   'height': 480,
   'width': 854},
  {'id': 'ffout-6/00852.jpg',
   'file_name':

In [9]:
with open('/data/flag/flag_coco_test.json','w') as f:
    json.dump(test_coco,f)

In [25]:
final_test_coco = convert_flag_to_coco('/data/flag/flag_ann_finaltest.json',image_prefix='/data/flag/test_dataset_label/images')

ffout-a0021so808b/00492.jpg
ffout-c002160nqjr/00664.jpg
ffout-i09381b3eeg/01480.jpg
ffout-i09381b3eeg/00733.jpg
ffout-c002160nqjr/01456.jpg
ffout-u00219eu8m3/00421.jpg
ffout-n002174wq4p/01797.jpg
ffout-n0021r9pa4t/01321.jpg
ffout-n002174wq4p/04277.jpg
ffout-f00215tk67q/01054.jpg
ffout-n0021k4xkub/01642.jpg
ffout-c0021j4eqth/00820.jpg
ffout-n0021k4xkub/01720.jpg
ffout-e00213twop9/02791.jpg
ffout-e00213twop9/00793.jpg
ffout-z00212bjkvp/03640.jpg
ffout-f00215tk67q/03154.jpg
ffout-i09381b3eeg/00166.jpg
ffout-z00212bjkvp/01072.jpg
ffout-y0021lu67r7/02011.jpg
ffout-e00213twop9/03121.jpg
ffout-c002160nqjr/00097.jpg
ffout-a0021so808b/00522.jpg
ffout-n002174wq4p/02253.jpg
ffout-c002160nqjr/02014.jpg
ffout-w0021n2d80c/00649.jpg
ffout-c002160nqjr/02095.jpg
ffout-i09381b3eeg/00952.jpg
ffout-h0021sst65l/00865.jpg
ffout-n0021k4xkub/00666.jpg
ffout-e00213twop9/00091.jpg
ffout-f00215tk67q/02962.jpg
ffout-f00215tk67q/03079.jpg
ffout-n002174wq4p/00326.jpg
ffout-n002174wq4p/03686.jpg
ffout-z00212bjkvp/00

ffout-i00211vjemz/00727.jpg
ffout-i09381b3eeg/01621.jpg
ffout-u00219eu8m3/00640.jpg
ffout-b00219wkkjc/04739.jpg
ffout-a0021so808b/00122.jpg
ffout-p00210wgixi/00391.jpg
ffout-n002174wq4p/04054.jpg
ffout-e00213twop9/04072.jpg
ffout-h0021sst65l/00184.jpg
ffout-a0021so808b/01830.jpg
ffout-n0021r9pa4t/01306.jpg
ffout-e00213twop9/00658.jpg
ffout-i00211vjemz/01831.jpg
ffout-b00219wkkjc/02569.jpg
ffout-y0021lu67r7/00856.jpg
ffout-i00211vjemz/04168.jpg
ffout-b00219wkkjc/04692.jpg
ffout-f00215tk67q/03145.jpg
ffout-n0021r9pa4t/00142.jpg
ffout-c002160nqjr/00598.jpg
ffout-c002160nqjr/01561.jpg
ffout-u00219eu8m3/00520.jpg
ffout-z00212bjkvp/00364.jpg
ffout-n0021k4xkub/01697.jpg
ffout-y0021lu67r7/02008.jpg
ffout-h0021sst65l/00295.jpg
ffout-u00219eu8m3/01240.jpg
ffout-n0021r9pa4t/00349.jpg
ffout-n0021r9pa4t/01042.jpg
ffout-n00217r6s3g/01842.jpg
ffout-a0021so808b/00417.jpg
ffout-i09381b3eeg/00544.jpg
ffout-i09381b3eeg/00721.jpg
ffout-u00219eu8m3/00496.jpg
ffout-a0021so808b/00777.jpg
ffout-u00219eu8m3/00

ffout-f00215tk67q/02023.jpg
ffout-n0021r9pa4t/01450.jpg
ffout-a0021so808b/01893.jpg
ffout-e00213twop9/00691.jpg
ffout-a0021so808b/01548.jpg
ffout-z00212bjkvp/00220.jpg
ffout-i09381b3eeg/00781.jpg
ffout-n002174wq4p/01398.jpg
ffout-n00217r6s3g/01997.jpg
ffout-c002160nqjr/00595.jpg
ffout-y0021lu67r7/00553.jpg
ffout-f00215tk67q/02278.jpg
ffout-n00217r6s3g/00085.jpg
ffout-f00215tk67q/01366.jpg
ffout-a0021so808b/00625.jpg
ffout-y0021lu67r7/02626.jpg
ffout-z00212bjkvp/01225.jpg
ffout-n00217r6s3g/00708.jpg
ffout-i00211vjemz/02881.jpg
ffout-n0021r9pa4t/00802.jpg
ffout-n0021r9pa4t/00040.jpg
ffout-f00215tk67q/03064.jpg
ffout-n0021r9pa4t/00169.jpg
ffout-z00212bjkvp/01207.jpg
ffout-p00210wgixi/00178.jpg
ffout-a0021so808b/01933.jpg
ffout-y0021lu67r7/00625.jpg
ffout-i09381b3eeg/00622.jpg
ffout-c0021j4eqth/00154.jpg
ffout-c002160nqjr/01189.jpg
ffout-p00210wgixi/00085.jpg
ffout-n0021r9pa4t/00019.jpg
ffout-n0021r9pa4t/00970.jpg
ffout-y0021lu67r7/01621.jpg
ffout-n00217r6s3g/04312.jpg
ffout-n0021r9pa4t/01

ffout-u00219eu8m3/00448.jpg
ffout-p00210wgixi/00796.jpg
ffout-f00215tk67q/02938.jpg
ffout-p00210wgixi/00832.jpg
ffout-n0021k4xkub/01166.jpg
ffout-n0021r9pa4t/01033.jpg
ffout-f00215tk67q/00892.jpg
ffout-a0021so808b/00838.jpg
ffout-i09381b3eeg/00346.jpg
ffout-i09381b3eeg/00283.jpg
ffout-n0021r9pa4t/00982.jpg
ffout-i09381b3eeg/01009.jpg
ffout-y0021lu67r7/00865.jpg
ffout-p00210wgixi/01108.jpg
ffout-n002174wq4p/03470.jpg
ffout-n002174wq4p/01552.jpg
ffout-z00212bjkvp/02587.jpg
ffout-f00215tk67q/01204.jpg
ffout-n0021k4xkub/00819.jpg
ffout-c002160nqjr/00781.jpg
ffout-c002160nqjr/02170.jpg
ffout-f00215tk67q/00415.jpg
ffout-n0021r9pa4t/00211.jpg
ffout-e00213twop9/02053.jpg
ffout-y0021lu67r7/01303.jpg
ffout-n00217r6s3g/00744.jpg
ffout-i00211vjemz/03430.jpg
ffout-u00219eu8m3/00148.jpg
ffout-c002160nqjr/01396.jpg
ffout-c002160nqjr/00529.jpg
ffout-h0021sst65l/02566.jpg
ffout-e00213twop9/02539.jpg
ffout-n002174wq4p/02371.jpg
ffout-u00219eu8m3/00142.jpg
ffout-e00213twop9/01804.jpg
ffout-n0021k4xkub/00

ffout-p00210wgixi/00283.jpg
ffout-y0021lu67r7/00469.jpg
ffout-w0021n2d80c/00040.jpg
ffout-f00215tk67q/03910.jpg
ffout-h0021sst65l/02644.jpg
ffout-f00215tk67q/03109.jpg
ffout-n002174wq4p/04489.jpg
ffout-n0021k4xkub/01741.jpg
ffout-h0021sst65l/00154.jpg
ffout-i09381b3eeg/01381.jpg
ffout-w0021n2d80c/00667.jpg
ffout-i00211vjemz/01309.jpg
ffout-e00213twop9/01942.jpg
ffout-p00210wgixi/00484.jpg
ffout-n002174wq4p/00498.jpg
ffout-u00219eu8m3/01591.jpg
ffout-n002174wq4p/03160.jpg
ffout-h0021sst65l/00577.jpg
ffout-n0021k4xkub/01700.jpg
ffout-z00212bjkvp/02914.jpg
ffout-y0021lu67r7/01519.jpg
ffout-n00217r6s3g/01191.jpg
ffout-y0021lu67r7/00157.jpg
ffout-e00213twop9/02629.jpg
ffout-i09381b3eeg/00832.jpg
ffout-z00212bjkvp/01960.jpg
ffout-n0021r9pa4t/00598.jpg
ffout-c0021j4eqth/00361.jpg
ffout-i09381b3eeg/00670.jpg
ffout-h0021sst65l/01795.jpg
ffout-u00219eu8m3/01183.jpg
ffout-e00213twop9/04447.jpg
ffout-n0021r9pa4t/00757.jpg
ffout-p00210wgixi/00988.jpg
ffout-i09381b3eeg/00907.jpg
ffout-f00215tk67q/00

In [26]:
final_test_coco

{'images': [{'id': 'ffout-a0021so808b/00492.jpg',
   'file_name': 'ffout-a0021so808b/00492.jpg',
   'height': 480,
   'width': 848},
  {'id': 'ffout-c002160nqjr/00664.jpg',
   'file_name': 'ffout-c002160nqjr/00664.jpg',
   'height': 480,
   'width': 848},
  {'id': 'ffout-i09381b3eeg/01480.jpg',
   'file_name': 'ffout-i09381b3eeg/01480.jpg',
   'height': 484,
   'width': 864},
  {'id': 'ffout-i09381b3eeg/00733.jpg',
   'file_name': 'ffout-i09381b3eeg/00733.jpg',
   'height': 484,
   'width': 864},
  {'id': 'ffout-c002160nqjr/01456.jpg',
   'file_name': 'ffout-c002160nqjr/01456.jpg',
   'height': 480,
   'width': 848},
  {'id': 'ffout-u00219eu8m3/00421.jpg',
   'file_name': 'ffout-u00219eu8m3/00421.jpg',
   'height': 480,
   'width': 848},
  {'id': 'ffout-n002174wq4p/01797.jpg',
   'file_name': 'ffout-n002174wq4p/01797.jpg',
   'height': 480,
   'width': 848},
  {'id': 'ffout-n0021r9pa4t/01321.jpg',
   'file_name': 'ffout-n0021r9pa4t/01321.jpg',
   'height': 480,
   'width': 848},
  {'id

In [28]:
with open('/data/flag/flag_coco_finaltest.json','w') as f:
    json.dump(final_test_coco,f)

In [29]:
annType = ['segm','bbox','keypoints']
annType = annType[1]      #specify type here
prefix = 'person_keypoints' if annType=='keypoints' else 'instances'
print('Running demo for *%s* results.'%(annType))

Running demo for *bbox* results.


In [35]:
#initialize COCO ground truth api
#dataDir='/data/flag'
#dataType='test'
#annFile = '%s/annotations/%s_%s.json'%(dataDir,prefix,dataType)
annFile = '/data/flag/flag_coco_finaltest.json'
cocoGt=COCO(annFile)

loading annotations into memory...
Done (t=0.24s)
creating index...
index created!


In [36]:
#initialize COCO detections api
#resFile='%s/results/%s_%s_fake%s100_results.json'
#resFile = resFile%(dataDir, prefix, dataType, annType)
resFile = '/data/Detection_proj/yolov5-master/runs/test/final_test7/best_predictions.json'
cocoDt=cocoGt.loadRes(resFile)

Loading and preparing results...
DONE (t=0.68s)
creating index...
index created!


In [5]:
imgIds=sorted(cocoGt.getImgIds())
imgIds=imgIds[0:100]
imgId = imgIds[np.random.randint(100)]

In [37]:
# running evaluation
cocoEval = COCOeval(cocoGt,cocoDt,annType)
#cocoEval.params.imgIds  = imgIds


In [38]:
cocoEval

In [50]:
pdb.set_trace()
cocoEval.evaluate()

--Return--
None
> <ipython-input-50-9e180d8dc841>(1)<module>()
----> 1 pdb.set_trace()
      2 cocoEval.evaluate()

ipdb> s
    [... skipped 1 hidden frame]

    [... skipped 1 hidden frame]

    [... skipped 1 hidden frame]

    [... skipped 1 hidden frame]

> /data/anaconda3/envs/map/lib/python3.7/site-packages/IPython/core/interactiveshell.py(3349)run_ast_nodes()
   3347                     to_run.append((node, 'single'))
   3348 
-> 3349                 for node,mode in to_run:
   3350                     if mode == 'exec':
   3351                         mod = Module([node], [])

ipdb> n
> /data/anaconda3/envs/map/lib/python3.7/site-packages/IPython/core/interactiveshell.py(3350)run_ast_nodes()
   3348 
   3349                 for node,mode in to_run:
-> 3350                     if mode == 'exec':
   3351                         mod = Module([node], [])
   3352                     elif mode == 'single':

ipdb> 
> /data/anaconda3/envs/map/lib/python3.7/site-packages/IPython/core/in

In [40]:
cocoEval.ious

{('ffout-a0021so808b/00122.jpg', 0): [],
 ('ffout-a0021so808b/00122.jpg', 1): [],
 ('ffout-a0021so808b/00122.jpg', 2): [],
 ('ffout-a0021so808b/00122.jpg', 3): [],
 ('ffout-a0021so808b/00122.jpg', 4): [],
 ('ffout-a0021so808b/00122.jpg', 5): [],
 ('ffout-a0021so808b/00122.jpg', 6): [],
 ('ffout-a0021so808b/00122.jpg', 7): [],
 ('ffout-a0021so808b/00122.jpg', 8): [],
 ('ffout-a0021so808b/00122.jpg', 9): [],
 ('ffout-a0021so808b/00122.jpg',
  10): array([[0.87005165, 0.        , 0.        ],
        [0.        , 0.64377959, 0.        ],
        [0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.55968617],
        [0.        , 0.        , 0.74256876],
        [0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        ],

In [45]:
cocoEval.params._gts

AttributeError: 'Params' object has no attribute '_gts'

In [ ]:
cocoEval.evaluate()
cocoEval.accumulate()
cocoEval.summarize()